In [1]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import pandas as pd
import numpy as np

In [13]:
TRAIN_DATA_PATH = 'E:/2018-2019AutumnTerm/2019/global_urban_computing/data'
all_y_pred = np.array([])
all_y_test = np.array([])
for i in range(81):
    train_df = pd.read_csv(TRAIN_DATA_PATH + '/data_feature/st_' + str(i) + '.csv')
    test_df = pd.read_csv(TRAIN_DATA_PATH + '/data_28/station_' + str(i) + '.csv')

    # Converting to date format
    train_df["timestamp"] = pd.to_datetime(train_df["timestamp"])
    test_df["timestamp"] = pd.to_datetime(test_df["timestamp"])

    train_y = train_df[['inNums', 'outNums']]
    test_y = test_df[['inNums', 'outNums']]
    # Processing Dates...
    train_df["Minute"] = train_df["timestamp"].apply(lambda x: x.minute)
    test_df["Minute"] = test_df["timestamp"].apply(lambda x: x.minute)
    train_df["DayOfMonth"] = train_df["timestamp"].apply(lambda x: x.day)
    test_df["DayOfMonth"] = test_df["timestamp"].apply(lambda x: x.day)

    train_df.drop(["timestamp", 'inNums', 'outNums', 'lag_yesterday_out', 'lag_yesterday_in', 'lag_last_week_out',
                   'lag_last_week_in'], axis=1, inplace=True)
    test_df.drop(["timestamp", 'inNums', 'outNums', 'lag_yesterday_out', 'lag_yesterday_in', 'lag_last_week_out',
                  'lag_last_week_in'], axis=1, inplace=True)
    params = {
            'learning_rate': 0.05,
            'n_estimators': 200,
            'subsample': 0.7,
            'colsample_bytree': 0.7,
            'max_depth': 15,
            'min_child_weight': 1.1,
            'reg_alpha': 0,
            'gamma': 0.1,
            'eval_metric':'mae',
    }

    # model
    max_depth = 30
    # regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100,
    #                                                           max_depth=max_depth,
    #                                                           random_state=0))
    regr_multirf = MultiOutputRegressor(xgb.XGBRegressor(learning_rate=params['learning_rate'], n_estimators=params['n_estimators'],
                             booster='gbtree', objective='reg:linear', n_jobs=-1, subsample=params['subsample'],
                             colsample_bytree=params['colsample_bytree'], random_state=0,
                             max_depth=params['max_depth'], gamma=params['gamma'],
                             min_child_weight=params['min_child_weight'], reg_alpha=params['reg_alpha'], eval_metric='mae'))
    regr_multirf.fit(np.array(train_df)[:, :], np.array(train_y)[:, :])
    y_multirf = regr_multirf.predict(np.array(test_df))
    if i == 0:
        all_y_pred = y_multirf
        all_y_test = np.array(test_y)
    else:
        all_y_pred = np.append(all_y_pred, y_multirf, axis=0)
        all_y_test = np.append(all_y_test, np.array(test_y), axis=0)

In [14]:
mean_absolute_error(all_y_test, all_y_pred)

13.954773826691973

In [8]:
df = pd.read_csv(TRAIN_DATA_PATH +  '/Metro_testA/testA_submit_2019-01-29.csv')
df.head()

,stationID,startTime,endTime,inNums,outNums
0,0,2019-01-29 00:00:00,2019-01-29 00:10:00,NaN,NaN
1,0,2019-01-29 00:10:00,2019-01-29 00:20:00,NaN,NaN
2,0,2019-01-29 00:20:00,2019-01-29 00:30:00,NaN,NaN
3,0,2019-01-29 00:30:00,2019-01-29 00:40:00,NaN,NaN
4,0,2019-01-29 00:40:00,2019-01-29 00:50:00,NaN,NaN


In [9]:
test_df = pd.read_csv(TRAIN_DATA_PATH + '/Metro_testA/testA_submit_2019-01-29.csv')
test_df.drop(["stationID", "endTime", "inNums", "outNums"], axis=1, inplace=True)
test_list = [None] * 81
for i in range(81):
    test_list[i] = test_df[i * 144:(i + 1) * 144]
type(test_list[80])

pandas.core.frame.DataFrame

In [10]:
train_df28 = pd.read_csv(TRAIN_DATA_PATH + '/data_28/station_6.csv')
train_df28.head()

,timestamp,inNums,outNums,week_day,current_hour,lag_yesterday_in,lag_yesterday_out,lag_last_week_in,lag_last_week_out,is_holiday
0,2019-01-28 00:00:00,0,2,0,0,1,2,0,0,0
1,2019-01-28 00:10:00,0,0,0,0,3,1,2,1,0
2,2019-01-28 00:20:00,0,0,0,0,3,3,0,0,0
3,2019-01-28 00:30:00,0,0,0,0,0,2,0,0,0
4,2019-01-28 00:40:00,0,0,0,0,2,1,0,0,0


In [15]:
all_y_pred = np.array([])
for i in range(81):
    train_df = pd.read_csv(TRAIN_DATA_PATH + '/data_feature/st_' + str(i) + '.csv')
    train_df28 = pd.read_csv(TRAIN_DATA_PATH + '/data_28/station_' + str(i) + '.csv')
    train_df.drop(['lag_yesterday_out', 'lag_yesterday_in', 'lag_last_week_out', 'lag_last_week_in'], axis=1,
                  inplace=True)
    train_df28.drop(['lag_yesterday_out', 'lag_yesterday_in', 'lag_last_week_out', 'lag_last_week_in'], axis=1,
                    inplace=True)
    train_df = train_df.append(train_df28)
    test_df = test_list[i]

    # Converting to date format
    train_df["timestamp"] = pd.to_datetime(train_df["timestamp"])
    test_df["timestamp"] = pd.to_datetime(test_df["startTime"])

    train_y = train_df[['inNums', 'outNums']]
    # Processing Dates...
    test_df['week_day'] = test_df['timestamp'].apply(lambda x: x.dayofweek)
    test_df['current_hour'] = test_df['timestamp'].apply(lambda x: x.hour)
    test_df['is_holiday'] = test_df['week_day'].isin([5, 6])
    test_df['is_holiday'] = test_df['is_holiday'] * 1

    train_df["Minute"] = train_df["timestamp"].apply(lambda x: x.minute)
    test_df["Minute"] = test_df["timestamp"].apply(lambda x: x.minute)
    train_df["DayOfMonth"] = train_df["timestamp"].apply(lambda x: x.day)
    test_df["DayOfMonth"] = test_df["timestamp"].apply(lambda x: x.day)

    train_df.drop(['timestamp', 'inNums', 'outNums'], axis=1, inplace=True)
    test_df.drop(['timestamp', "startTime"], axis=1, inplace=True)

    params = {
        'learning_rate': 0.05,
        'n_estimators': 200,
        'subsample': 0.6,
        'colsample_bytree': 0.6,
        'max_depth': 15,
        'min_child_weight': 1,
        'reg_alpha': 0,
        'gamma': 0.1,
        'eval_metric':'mae'
    }

    regr_multirf = MultiOutputRegressor(xgb.XGBRegressor(learning_rate=params['learning_rate'], n_estimators=params['n_estimators'],
                         booster='gbtree', objective='reg:linear', n_jobs=-1, subsample=params['subsample'],
                         colsample_bytree=params['colsample_bytree'], random_state=0,
                         max_depth=params['max_depth'], gamma=params['gamma'],
                         min_child_weight=params['min_child_weight'], reg_alpha=params['reg_alpha'], eval_metric='mae'))
    regr_multirf.fit(np.array(train_df)[:, :], np.array(train_y)[:, :])
    y_multirf = regr_multirf.predict(np.array(test_df))
    if i == 0:
        all_y_pred = y_multirf
    else:
        all_y_pred = np.append(all_y_pred, y_multirf, axis=0)
all_y_pred.shape

(11664, 2)

In [16]:
output= pd.DataFrame({"inNums":all_y_pred[:,0],"outNums":all_y_pred[:,1]})
output.head()

,inNums,outNums
0,2.262028,1.083293
1,-0.303423,2.621593
2,3.309143,0.106997
3,0.623737,-0.340769
4,-0.417611,1.138361


In [17]:
test_df = pd.read_csv(TRAIN_DATA_PATH +  '/Metro_testA/testA_submit_2019-01-29.csv')
test_df[["inNums","outNums"]]=output[["inNums","outNums"]]
test_df.head()

,stationID,startTime,endTime,inNums,outNums
0,0,2019-01-29 00:00:00,2019-01-29 00:10:00,2.262028,1.083293
1,0,2019-01-29 00:10:00,2019-01-29 00:20:00,-0.303423,2.621593
2,0,2019-01-29 00:20:00,2019-01-29 00:30:00,3.309143,0.106997
3,0,2019-01-29 00:30:00,2019-01-29 00:40:00,0.623737,-0.340769
4,0,2019-01-29 00:40:00,2019-01-29 00:50:00,-0.417611,1.138361


In [18]:
test_df.to_csv("E:/2018-2019AutumnTerm/2019/global_urban_computing/data/res/sub_03277.csv", index=False)